In [1]:
import pandas as pd
import random
from math import sqrt, log

# Load datasets
def load_data():
    movie_df = pd.read_csv('movie.csv')
    rating_df = pd.read_csv('rating.csv')
    tag_df = pd.read_csv('tag.csv')
    genome_tags_df = pd.read_csv('genome_tags.csv')
    return movie_df, rating_df, tag_df, genome_tags_df

# Function to merge chunks of DataFrames
def merge_chunks(chunks):
    merged_df = pd.DataFrame()
    for chunk in chunks:
        merged_df = pd.concat([merged_df, chunk])
    return merged_df

# Load and merge datasets
def load_and_merge_data():
    movie_df, rating_df, tag_df, genome_tags_df = load_data()
    
    # Merge movie, rating, tag, and genome_tags datasets
    merged_df = pd.merge(movie_df, rating_df, on='movieId')
    merged_df = pd.merge(merged_df, tag_df, on='movieId')
    merged_df = pd.merge(merged_df, genome_tags_df, on='movieId')
    
    return merged_df

# Node class for the MCTS algorithm
class Node:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []
        self.visits = 0
        self.reward = 0

    def select_child(self):
        if not self.children:
            return None
        max_child = None
        max_score = float('-inf')
        for child in self.children:
            if child.visits == 0:
                score = float('inf')  # Set score to infinity if child has not been visited
            else:
                score = child.reward / child.visits + sqrt(2 * log(self.visits) / child.visits)
            if score > max_score:
                max_score = score
                max_child = child
        return max_child

    def expand(self, action_space):
        for action in action_space:
            self.children.append(Node(action, parent=self))

    def update(self, reward):
        self.visits += 1
        self.reward += reward
        if self.parent:
            self.parent.update(reward)

# Monte Carlo Tree Search (MCTS) algorithm
class MCTS:
    def __init__(self, root_state, action_space):
        self.root = Node(root_state)
        self.action_space = action_space

    def search(self, num_simulations):
        for _ in range(num_simulations):
            node = self.root
            while node.children:
                node = node.select_child()
            if node.visits > 0:
                node.expand(self.action_space)
                node = random.choice(node.children)
            reward = 1  # Placeholder reward for demonstration
            while node:
                node.update(reward)
                node = node.parent

# Main function
def main():
    # Load and merge datasets
    movies_df = load_and_merge_data()
    
    # Main loop for recommending movies
    while True:
        # Assuming user preferences are related to genres
        user_genre = input("Enter your preferred movie genre: ").lower()
        
        # Filter movies based on user's preferred genre
        movies_in_genre = movies_df[movies_df['genres'].str.lower().str.contains(user_genre)]
        
        if movies_in_genre.empty:
            print("Sorry, there are no movies available in the selected genre. Please choose another genre.")
            continue
        
        # Initialize MCTS with a random movie from the selected genre
        root_state = random.choice(movies_in_genre['title'].tolist())
        mcts = MCTS(root_state, movies_in_genre['title'].tolist())
        
        # Perform MCTS search
        num_simulations = 10000
        mcts.search(num_simulations)
        
        # Get recommended movie based on MCTS results
        recommended_movie_node = max(mcts.root.children, key=lambda child: child.reward / child.visits)
        recommended_movie = recommended_movie_node.state
        print("Recommended Movie:", recommended_movie)
        
        # Ask user for feedback
        seen_movie = input(f"Have you seen '{recommended_movie}'? (yes/no): ").lower()
        if seen_movie == 'no':
            continue
        else:
            another_recommendation = input("Do you want another recommendation? (yes/no): ").lower()
            if another_recommendation != 'yes':
                break

if __name__ == "__main__":
    main()


FileNotFoundError: [Errno 2] No such file or directory: 'movie.csv'